In [2]:
import pandas as pd

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .getOrCreate()

In [9]:
spark

In [4]:
path = "data-files/customers-tab-delimited/part-m-00000"

In [5]:
from pyspark.sql import types

schema = types.StructType([
types.StructField('id', types.StringType(), True),
types.StructField('first_name', types.StringType(), True), 
types.StructField('last_name', types.StringType(), True), 
types.StructField('email', types.StringType(), True), 
types.StructField('password', types.StringType(), True),
types.StructField('street', types.StringType(), True),
types.StructField('city', types.StringType(), True), 
types.StructField('state', types.StringType(), True),
types.StructField('zipcode', types.StringType(), True)

])

In [6]:
df =  spark.read.format('csv')\
.schema(schema)\
.option('sep', r'\t')\
.load(path)

## Trial with pandas

In [187]:
df_pandas= df.toPandas()

In [188]:
df_pandas.head()

,id,first_name,last_name,email,password,street,city,state,zipcode
0,1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
1,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
2,3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725
3,4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
4,5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,10 Crystal River Mall,Caguas,PR,00725


In [189]:
ls = df_pandas['state'].unique().tolist()

In [190]:
sorted(ls)

['AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DC',
 'DE',
 'FL',
 'GA',
 'HI',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MA',
 'MD',
 'MI',
 'MN',
 'MO',
 'MT',
 'NC',
 'ND',
 'NJ',
 'NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'OR',
 'PA',
 'PR',
 'RI',
 'SC',
 'TN',
 'TX',
 'UT',
 'VA',
 'WA',
 'WI',
 'WV']

In [191]:
df_pandas[df_pandas['state'] == 'CA']

,id,first_name,last_name,email,password,street,city,state,zipcode
3,4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
13,14,Katherine,Smith,XXXXXXXXX,XXXXXXXXX,5666 Hazy Pony Square,Pico Rivera,CA,90660
14,15,Jane,Luna,XXXXXXXXX,XXXXXXXXX,673 Burning Glen,Fontana,CA,92336
17,18,Robert,Smith,XXXXXXXXX,XXXXXXXXX,2734 Hazy Butterfly Circle,Martinez,CA,94553
34,35,Margaret,Wright,XXXXXXXXX,XXXXXXXXX,9456 Sleepy Jetty,Oceanside,CA,92056
...,...,...,...,...,...,...,...,...,...
12400,12401,Angela,Gray,XXXXXXXXX,XXXXXXXXX,109 Hazy Heath,Los Angeles,CA,90033
12402,12403,Richard,Ferguson,XXXXXXXXX,XXXXXXXXX,5382 Hazy Pathway,Cypress,CA,90630
12422,12423,Stephen,Smith,XXXXXXXXX,XXXXXXXXX,3445 Harvest Campus,Palmdale,CA,93550
12423,12424,Judy,Phillips,XXXXXXXXX,XXXXXXXXX,4534 Cinder Concession,San Diego,CA,92111


In [192]:
from pyspark.sql.functions import col
df.select(col("first_name"),col("last_name")).show()

+-----------+---------+
| first_name|last_name|
+-----------+---------+
|    Richard|Hernandez|
|       Mary|  Barrett|
|        Ann|    Smith|
|       Mary|    Jones|
|     Robert|   Hudson|
|       Mary|    Smith|
|    Melissa|   Wilcox|
|      Megan|    Smith|
|       Mary|    Perez|
|    Melissa|    Smith|
|       Mary|  Huffman|
|Christopher|    Smith|
|       Mary|  Baldwin|
|  Katherine|    Smith|
|       Jane|     Luna|
|    Tiffany|    Smith|
|       Mary| Robinson|
|     Robert|    Smith|
|  Stephanie| Mitchell|
|       Mary|    Ellis|
+-----------+---------+
only showing top 20 rows



## Start Here

In [8]:
result = df.filter(df.state == "CA")

In [194]:
result.columns

['id',
 'first_name',
 'last_name',
 'email',
 'password',
 'street',
 'city',
 'state',
 'zipcode']

In [9]:
from pyspark.sql import functions
from pyspark.sql.functions import concat


In [10]:
from pyspark.sql import functions as F
result=result.withColumn("row",concat( \
    F.col('first_name'), F.lit(' '), \
    F.col('last_name')))

In [11]:
result = result.select(F.col('row'))

In [12]:
result.show(truncate=False)

+---------------+
|row            |
+---------------+
|Mary Jones     |
|Katherine Smith|
|Jane Luna      |
|Robert Smith   |
|Margaret Wright|
|Mary Smith     |
|Howard Smith   |
|Mary Kim       |
|Douglas James  |
|Mary Simmons   |
|Frank Gillespie|
|Joseph Young   |
|Sean Smith     |
|Lauren Freeman |
|Alice Warner   |
|Mary Smith     |
|Mary Gallagher |
|Daniel Maxwell |
|Shirley Mcclain|
|Mary Smith     |
+---------------+
only showing top 20 rows



In [199]:
result.write.mode('Overwrite').text('result/scenario1/solution')